In [2]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [3]:
sp = SparkSession.builder.appName("Checker").getOrCreate()
sp

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 23:08:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 23:08:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [23]:
transactions = sp.read.option("inferSchema", True).parquet("../data/curated/transactions")
transactions.show(5)
# There are 4422 merchants
# There are 14195505 transactions

+-------+------------+------------+--------------+-----------+
|user_id|merchant_abn|dollar_value|order_datetime|   order_id|
+-------+------------+------------+--------------+-----------+
|  14935| 79417999332|      136.07|    2021-11-26|68719476736|
|      1| 46451548968|       72.62|    2021-11-26|68719476737|
|  14936| 89518629617|        3.08|    2021-11-26|68719476738|
|      1| 49167531725|       51.58|    2021-11-26|68719476739|
|  14936| 31101120643|       25.23|    2021-11-26|68719476740|
+-------+------------+------------+--------------+-----------+
only showing top 5 rows



+------------+-----+--------------------+
|merchant_abn|count|      Applied_Column|
+------------+-----+--------------------+
| 83412691377|14288|0.001006515794964...|
| 38700038932| 7132|5.024125594686487E-4|
| 35344855546| 1522|1.072170380694452...|
| 15613631617| 1785|1.257440295361102E-4|
| 19839532017|  726|5.114294982813221...|
| 24406529929| 4184|2.947411874392633...|
| 73256306726| 5263|3.707511638367215...|
| 92202115241|  120|8.453380136881357E-6|
| 73841664453|  959|6.755659626057686E-5|
| 98345415950|    8|5.635586757920905E-7|
| 41956465747|  279|1.965410881824915...|
| 12516851436|  210|1.479341523954237...|
| 78916025936|   67|4.719803909758758E-6|
| 48214071373|  540|3.804021061596611E-5|
| 52763133264|  125|8.805604309251414E-6|
| 60654402457|  199|1.401852206032825...|
| 56395390867|   47|3.310907220278532E-6|
| 34440496342|  215|1.514563941191243...|
| 96946925998|  131|9.228273316095483E-6|
| 37935728745|   14|9.862276826361584E-7|
+------------+-----+--------------

In [ ]:
# Average Transaction Count
number_of_transactions_by_merchant = transactions.groupBy("merchant_abn").count()
number_of_transactions_by_merchant = number_of_transactions_by_merchant.withColumn("Average Customer Count",col("count")/transactions.count()).show()


In [44]:
# Average dollar sum value for each merchant /number of customers that merchant has

number_of_total_dollar_value = transactions.groupBy("merchant_abn").sum("dollar_value")
number_of_total_dollar_value = number_of_total_dollar_value.withColumnRenamed("merchant_abn","merchant_id")
number_of_average_dollar_value = number_of_transactions_by_merchant.join(number_of_total_dollar_value,number_of_total_dollar_value.merchant_id == number_of_users_by_merchant.merchant_abn,"inner")
number_of_average_dollar_value = number_of_average_dollar_value.withColumn("Average dollar value per number of customers",col("sum(dollar_value)")/col("count"))
number_of_average_dollar_value = number_of_average_dollar_value.drop(number_of_average_dollar_value.merchant_id)
number_of_average_dollar_value = number_of_average_dollar_value.drop(number_of_average_dollar_value['count'])
number_of_average_dollar_value = number_of_average_dollar_value.drop(number_of_average_dollar_value['sum(dollar_value)'])
number_of_average_dollar_value.show(5)




+------------+--------------------------------------------+
|merchant_abn|Average dollar value per number of customers|
+------------+--------------------------------------------+
| 83412691377|                            34.8920058642805|
| 38700038932|                          1338.5004544450671|
| 35344855546|                           88.52646502010455|
| 15613631617|                          304.21872829415884|
| 19839532017|                                       157.0|
+------------+--------------------------------------------+
only showing top 5 rows



In [83]:
# Number of distinct dates each merchant 
number_of_dates_by_merchant = transactions.groupBy('merchant_abn','order_datetime').count()
number_of_dates_by_merchant = number_of_dates_by_merchant.drop('order_datetime')
number_of_dates_by_merchant = number_of_dates_by_merchant.groupBy('merchant_abn').sum('count')
number_of_dates_by_merchant = number_of_dates_by_merchant.withColumnRenamed('sum(count)','Number of distinct dates')
number_of_dates_by_merchant.show(5)


+------------+------------------------+
|merchant_abn|Number of distinct dates|
+------------+------------------------+
| 73841664453|                     959|
| 19839532017|                     726|
| 38700038932|                    7132|
| 35344855546|                    1522|
| 73256306726|                    5263|
+------------+------------------------+
only showing top 5 rows



14195505

In [81]:
# Number of Users for each merchant

number_of_users_for_each_merchant = transactions.groupBy('merchant_abn','user_id').count()
number_of_users_for_each_merchant = number_of_users_for_each_merchant.drop('user_id')
number_of_users_for_each_merchant = number_of_users_for_each_merchant.groupBy('merchant_abn').sum('count')
number_of_users_for_each_merchant = number_of_users_for_each_merchant.withColumnRenamed('sum(count)','Number of users for each merchant')
number_of_users_for_each_merchant.show(5)

22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:36 WARN RowBasedKeyValueBatch: Calling spill() on

22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:38 WARN RowBasedKeyValueBatch: Calling spill() on

22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:43 WARN RowBasedKeyValueBatch: Calling spill() on

22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:44 WARN RowBasedKeyValueBatch: Calling spill() on

+------------+---------------------------------+
|merchant_abn|Number of users for each merchant|
+------------+---------------------------------+
| 12516851436|                              210|
| 15613631617|                             1785|
| 19839532017|                              726|
| 24406529929|                             4184|
| 28767881738|                                4|
+------------+---------------------------------+
only showing top 5 rows

22/10/06 01:38:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:47 WARN RowBasedKeyValueBatch: Calling spill() on R

22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:48 WARN RowBasedKeyValueBatch: Calling spill() on

22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:53 WARN RowBasedKeyValueBatch: Calling spill() on

22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/10/06 01:38:54 WARN RowBasedKeyValueBatch: Calling spill() on

14195505